In [124]:
# Import
import sys
import os
import shutil
import numpy as np
from pprint import pprint, pformat
import math
import copy
import time
import random
import pandas as pd
from sklearn import datasets
import plotly.express as px
import plotly.graph_objects as go
import calendar

In [125]:
np.seterr('raise', under='print', over='print')

{'divide': 'raise', 'over': 'print', 'under': 'print', 'invalid': 'raise'}

In [126]:
clean_history_result = False
if clean_history_result:
    shutil.rmtree('./STARE-result')

In [127]:
log_path = './STARE-result/' + time.strftime("%Y-%m-%d--%H-%M", time.localtime())

os.makedirs(log_path, exist_ok=True)

def log(msg):
    print(msg)
    sys.stdout.flush()
    fp = open(log_path + '/output.log', mode='a')
    fp.write(msg + "\n")
    fp.close()


In [128]:
df = pd.read_csv('big_data_new.csv')
df

,Month,Day,Hour,1995,1996,1997,1998,1999,2000,2001,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,January,1,0,13.225006,18.162506,15.475006,19.350006,15.225006,12.019012,17.600006,...,14.475006,16.725006,11.350006,19.850006,12.600006,18.225006,12.162506,16.662506,11.350006,17.162506
1,January,1,1,12.850006,17.350006,15.725006,18.600006,14.475006,12.048553,16.475006,...,13.725006,15.912506,11.412506,18.787506,11.600006,16.412506,11.600006,15.287506,10.975006,16.037506
2,January,1,2,12.412506,16.725006,15.725006,17.912506,14.850006,12.234039,17.412506,...,13.787506,14.287506,9.412506,15.912506,10.975006,14.975006,10.412506,13.787506,10.600006,14.850006
3,January,1,3,11.287506,15.037506,14.225006,15.412506,11.975006,10.198364,12.975006,...,12.412506,12.162506,7.975006,14.225006,10.287506,14.162506,8.850006,12.225006,9.600006,13.912506
4,January,1,4,11.287506,14.787506,13.787506,15.662506,11.850006,10.057220,12.975006,...,11.100006,11.350006,7.600006,14.225006,8.975006,12.725006,7.725006,11.537506,9.287506,13.225006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,December,31,19,16.287506,13.975006,17.725006,14.100006,11.100006,15.600006,13.350006,...,14.787506,9.662506,18.100006,11.600006,17.537506,10.537506,14.662506,12.912506,16.037506,15.287506
8756,December,31,20,17.350006,14.037506,18.162506,14.287506,11.475006,16.350006,13.537506,...,15.912506,10.162506,20.037506,12.600006,18.787506,11.225006,15.975006,12.725006,17.100006,16.100006
8757,December,31,21,19.975006,14.162506,20.350006,17.662506,12.725006,19.975006,15.662506,...,17.662506,11.225006,21.037506,13.662506,19.662506,11.850006,17.037506,12.100006,17.662506,16.975006
8758,December,31,22,18.475006,16.037506,20.412506,15.725006,12.225006,18.037506,13.850006,...,17.600006,11.350006,22.350006,13.225006,19.225006,12.475006,17.225006,12.100006,17.600006,16.412506


In [129]:
month_dict = {month: index for index, month in enumerate(calendar.month_abbr) if month}

def datetime_to_hours(month, day, hour):
    m = month_dict[month[:3]]
    d = 0
    if m == 1:      #Jan 31
        d = day
    elif m == 2:    #Feb
        d = 31 + day
    elif m == 3:    #Mar
        d = 59 + day
    elif m == 4:    #Apr
        d = 90 + day
    elif m == 5:    #May
        d = 120 + day
    elif m == 6:    #Jun
        d = 151 + day
    elif m == 7:    #Jul
        d = 181 + day
    elif m == 8:    #Aug
        d = 212 + day
    elif m == 9:    #Sep
        d = 243 + day
    elif m == 10:    #Oct
        d = 273 + day
    elif m == 11:   #Nov
        d = 304 + day
    elif m == 12:   #Dec
        d = 334 + day
    h = (d - 1) * 24 + hour
    return h
     

In [130]:
# Data stream generation
recent_years = 3
time_axis_scale = 35

month_dict = {month: index for index, month in enumerate(calendar.month_abbr) if month}

def time_series_gen():
    global anomaly_count
    
    for y in range(2019 - recent_years, 2019):
        for r in df.iterrows():
            yield datetime_to_hours(r[1]['Month'][:3], r[1]['Day'], r[1]['Hour']) / 8760 * time_axis_scale
            yield r[1][str(y)]
            yield 0

data = np.fromiter(time_series_gen(), float, 8760 * recent_years * 3).reshape((8760 * recent_years, 3))
log('Data stream from {0} to 2018, time axis scaling factor: {1}'.format(2019 - recent_years, time_axis_scale))


Data stream from 2016 to 2018, time axis scaling factor: 35


In [131]:
generate_anomaly = True
anomaly_rate = 0.003
anomaly_func = lambda orig: orig + random.uniform(2, 5) * random.choice([-1, 1])
anomaly_count = int(anomaly_rate * data.shape[0])

for i in np.random.randint(0, data.shape[0], anomaly_count):
    data[i, 1] = anomaly_func(data[i, 1])
    data[i, 2] = 1

if generate_anomaly:
    log('{0} anomaly generated'.format(anomaly_count))


78 anomaly generated


In [132]:
df = pd.DataFrame(data, columns=['time', 'value', 'label'])
fig = px.scatter(df)
fig.write_html(log_path + "/data.html")

In [133]:
# Data statistic

data_bound = data.min(axis=0, initial=None), data.max(axis=0, initial=None)
data_range = data_bound[1] - data_bound[0]
log(f"Data loaded: {data.shape}\n\
Min value: {data_bound[0]}\n\
Max value: {data_bound[1]}\n\
Range: {data_range}")

Data loaded: (26280, 3)
Min value: [0.       4.412506 0.      ]
Max value: [34.99600457 38.162506    1.        ]
Range: [34.99600457 33.75        1.        ]


In [134]:
# Parameters
windows_size = 8760 * 2

slide_size = 24 * 31

diagnal_cell_size = .2

error_allowance_threshold = 0.1

neighbor_kernel_number = 50

fixed_outlier_number = 30

fixed_outlier_score_threshold = 0

def kernel_function (u, h):
    try:
        val = (1.0 / (h * math.sqrt(2 * math.pi))) * math.exp(-0.5 * pow(u, 2) / pow(h, 2))
        return val
    except FloatingPointError:
        return sys.float_info.min
    
log('Window size: {0}'.format(windows_size))
log('Slide size: {0}'.format(slide_size))
log('Cell size: {0}'.format(diagnal_cell_size))
log('Error allowance: {0}'.format(error_allowance_threshold))
log('k Nearest neighbor: {0}'.format(neighbor_kernel_number))
log('Fixed outlier number: {0}'.format(fixed_outlier_number))
        

Window size: 17520
Slide size: 744
Cell size: 0.2
Error allowance: 0.1
k Nearest neighbor: 50
Fixed outlier number: 30


In [135]:
# Globals

data_pointer = 0

data_dimension = data.shape[1] - 1

data_records = dict()

cell_size = diagnal_cell_size / math.sqrt(data_dimension)

kernel_centers = dict()

kernel_centers_new = list()

kernel_centers_required_update = list()

kernel_centers_candidate_outliers = list()

window = list()

window_count = 0

expected_outlier_number = 0


true_outlier_number = 0


true_outliers = list()

In [136]:
class DataRecord:
    def __init__(self, value):
        self.key = tuple(value)
        self.array = np.array(value)
        self.index = int(data_pointer)
        self.created_window = int(window_count)
        self.last_update_window = -10
        self.kernel_center = None
        self.is_outlier = False
        self.outlier_score = 0
        self.local_density = sys.float_info.max
        self.previous_local_density = None

    def __str__(self):
        return str(self.key)

    def update_local_density(self):
        bandwidth = [0.0] * data_dimension

        for dim in range(data_dimension):
            dist = 0
            for kc in kernel_centers[self.kernel_center].nearest_kernel_centers:
                dist += abs(self.array[dim] - kernel_centers[kc].array[dim]) * kernel_centers[kc].weight / kernel_centers[self.kernel_center].nearest_kernel_centers_weight_sum
                
            bandwidth[dim] = max(dist, 0.001)

        density = 0
        
        for kc in kernel_centers[self.kernel_center].nearest_kernel_centers:
            product = 1
            for dim in range(data_dimension):
                product *= kernel_function(abs(self.array[dim] - kernel_centers[kc].array[dim]), bandwidth[dim])
            
            density += (kernel_centers[kc].weight / kernel_centers[self.kernel_center].nearest_kernel_centers_weight_sum) * product
                    
        self.local_density = density

In [137]:
class KernelCenter:
    def __init__(self, value):
        self.key = tuple(value)
        self.array = np.array(value)
        self.cell = hash_to_cell(self.array)
        self.data = list()
        self.weight = 0
        self.local_density = 0
        self.local_density_previous = 0
        self.local_density_upper_bound = 0
        self.local_density_lower_bound = 0
        self.local_density_mean = 0
        self.local_density_sd = 0
        self.bandwidth = list()
        self.nearest_kernel_centers = list()
        self.nearest_kernel_centers_weight_sum = 0
        self.nearest_kernel_centers_max_distance = 0
        self.outlier_score = 0
        self.outlier_score_upper_bound = 0
        self.outlier_score_lower_bound = 0
        self.cumulated_error = 0
        self.removed = False

    def __str__(self):
        return str(self.key)

    def update_nearest_kernel_centers(self):
        self.nearest_kernel_centers.clear()
        nearest_kcs = find_nearest_kernel_centers(self.key)
        self.nearest_kernel_centers_max_distance = 0
        self.nearest_kernel_centers_weight_sum = 0
        for kc in nearest_kcs:
            self.nearest_kernel_centers.append(kc[0])
            if kc[1] > self.nearest_kernel_centers_max_distance:
                self.nearest_kernel_centers_max_distance = kc[1]
            self.nearest_kernel_centers_weight_sum += kernel_centers[kc[0]].weight


    def update_bandwidth(self):
        self.bandwidth = [0.0] * data_dimension

        for dim in range(data_dimension):
            dist = 0
            for kc in self.nearest_kernel_centers:
                dist += abs(self.array[dim] - kernel_centers[kc].array[dim]) * kernel_centers[kc].weight / self.nearest_kernel_centers_weight_sum
                
            self.bandwidth[dim] = max(dist, 0.001)

    def update_local_density(self):
        # prev density
        self.local_density_previous = self.local_density

        # density
        density = 0
        
        for kc in self.nearest_kernel_centers:
            product = 1
            for dim in range(data_dimension):
                product *= kernel_function(abs(self.array[dim] - kernel_centers[kc].array[dim]), self.bandwidth[dim])
            density += (kernel_centers[kc].weight / self.nearest_kernel_centers_weight_sum) * product
                    
        self.local_density = density

        # density upper bound
        density = 0
        
        for kc in self.nearest_kernel_centers:
            product = 1
            for dim in range(data_dimension):
                dist = abs(self.array[dim] - kernel_centers[kc].array[dim]) - diagnal_cell_size / (2 * math.sqrt(data_dimension))
                if dist < 0:
                    dist = abs(self.array[dim] - kernel_centers[kc].array[dim])
                product *= kernel_function(dist, self.bandwidth[dim])
            density += (kernel_centers[kc].weight / self.nearest_kernel_centers_weight_sum) * product

        self.local_density_upper_bound = density

        # density lower bound
        density = 0

        for kc in self.nearest_kernel_centers:
            product = 1
            for dim in range(data_dimension):
                product *= kernel_function(abs(self.array[dim] - kernel_centers[kc].array[dim]) + diagnal_cell_size / (2 * math.sqrt(data_dimension)), self.bandwidth[dim])
            density += (kernel_centers[kc].weight / self.nearest_kernel_centers_weight_sum) * product

        self.local_density_lower_bound = density


    def update_outlier_score(self):
        neighbor_local_densities = [kernel_centers[nkc].local_density for nkc in self.nearest_kernel_centers]

        self.local_density_mean = np.mean(neighbor_local_densities)
        self.local_density_sd = np.std(neighbor_local_densities)

        if self.local_density_sd == 0:
            self.local_density_sd = np.finfo(np.float).tiny



        self.outlier_score = (self.local_density - self.local_density_mean) / self.local_density_sd
        self.outlier_score_upper_bound = (self.local_density_upper_bound - self.local_density_mean) / self.local_density_sd
        self.outlier_score_lower_bound = (self.local_density_lower_bound - self.local_density_mean) / self.local_density_sd
        

In [138]:
# Hash functions
def hash_to_cell(data_point):
    if type(data_point) is DataRecord:
        data_point = data_point.array
    return tuple((data_point - data_bound[0][:data_dimension]) // cell_size)


def hash_to_kernel_center(data_point):
    if type(data_point) is DataRecord:
        data_point = data_point.array
    cell = (data_point - data_bound[0][:data_dimension]) // cell_size
    return tuple(data_bound[0][:data_dimension] + cell * cell_size + cell_size / 2)

In [139]:
def distance_to_kernel_center(point, kernel_center):
    point = np.array(point)
    return np.linalg.norm(point - kernel_centers[kernel_center].array)

def find_nearest_kernel_centers(data_point):
    distance = dict()
    for kc in kernel_centers:
        if kernel_centers[kc].removed:
            continue
        distance[kc] = distance_to_kernel_center(data_point, kc)
    sorted_distance = sorted(distance.items(), key=lambda item: item[1])
    return sorted_distance[0:neighbor_kernel_number]

In [140]:
# Window sliding
def feed_new_slide():
    global data_pointer, expected_outlier_number, true_outlier_number

    new_weight_grid = dict()

    kernel_centers_new.clear()

    slide_start_time = data[data_pointer][0]

    for i in range(0, slide_size):
        data_record = DataRecord(data[data_pointer][:data_dimension])
        if data[data_pointer][-1] > 0:
            data_record.is_outlier = True
            expected_outlier_number += 1
            true_outlier_number += 1
            true_outliers.append(data_record)
        window.append(data_record)
        kc = hash_to_kernel_center(data_record)
        data_record.kernel_center = kc
        if kc not in new_weight_grid:
            new_weight_grid[kc] = 0
        if kc in kernel_centers and kernel_centers[kc].removed:
            kernel_centers.pop(kc)
        if kc not in kernel_centers:
            kernel_centers[kc] = KernelCenter(kc)
            kernel_centers_new.append(kc)
        kernel_centers[kc].data.append(data_record)
        new_weight_grid[kc] += 1
        data_pointer += 1
    
    log('Slide in: {0:.4f} to {1:.4f}'.format(slide_start_time, data[data_pointer - 1][0]))

    return new_weight_grid
    
def expire_old_slide():
    global expected_outlier_number, true_outlier_number

    slide_start_time = window[0].array[0]

    expired_weight_grid = dict()

    for i in range(0, slide_size):
        data_record = window.pop(0)
        if data_record.is_outlier:
            expected_outlier_number -= 1
            true_outlier_number -= 1
            true_outliers.remove(data_record)
        kc = hash_to_kernel_center(data_record)
        if kc not in expired_weight_grid:
            expired_weight_grid[kc] = 0
        kernel_centers[kc].data.remove(data_record)
        expired_weight_grid[kc] += 1

        slide_last_time = data_record.array[0]

    
    log('Slide out: {0:.4f} to {1:.4f}'.format(slide_start_time, slide_last_time))

    return expired_weight_grid

In [141]:
# Algorithm 2 UpdateDistribution - functions


def weight_grid_changes(new_weight_grid, expired_weight_grid):
    delta_grid = dict(new_weight_grid)
    for kc in expired_weight_grid:
        if kc not in delta_grid:
            delta_grid[kc] = 0
        delta_grid[kc] -= expired_weight_grid[kc]
        if delta_grid[kc] == 0:
            delta_grid.pop(kc)
    return delta_grid

def apply_weight_grid_change(delta_grid):
    global kernel_centers

    for kc in delta_grid:
        kernel_centers[kc].weight += delta_grid[kc]
        if kernel_centers[kc].weight == 0:
            kernel_centers[kc].removed = True
            # ↓Testing
            kernel_centers.pop(kc)

In [142]:
# Algorithm 2 UpdateDistribution - invocation
def update_distribution(new_weight_grid, expired_weight_grid):
    delta_grid = weight_grid_changes(new_weight_grid, expired_weight_grid)
    apply_weight_grid_change(delta_grid)
    return delta_grid

In [143]:
# Algo3
def update_changed_kernel_centers():
    for kc in kernel_centers_required_update:
        kernel_centers[kc].update_nearest_kernel_centers()
        kernel_centers[kc].update_bandwidth()
        kernel_centers[kc].update_local_density()

    for kc in kernel_centers_required_update:
        kernel_centers[kc].update_outlier_score()
        

In [144]:
# Algo4
def detect_local_outliers_cells():
    global kernel_centers_candidate_outliers
    kernel_centers_candidate_outliers.clear()
    kernel_center_scores = dict()
    for kc in kernel_centers:
        if kernel_centers[kc].removed:
            continue
        kernel_center_scores[kc] = kernel_centers[kc].outlier_score_upper_bound
        if len(kernel_centers_candidate_outliers) < expected_outlier_number:
            kernel_centers_candidate_outliers.append(kc)
        elif expected_outlier_number > 0:
            head_candidate_kernel = kernel_centers[kernel_centers_candidate_outliers[0]]
            if kernel_centers[kc].outlier_score_lower_bound > head_candidate_kernel.outlier_score_upper_bound:
                continue
            elif kernel_centers[kc].outlier_score_upper_bound < head_candidate_kernel.outlier_score_lower_bound:
                kernel_centers_candidate_outliers.pop(0)
                kernel_centers_candidate_outliers.append(kc)
            else:
                kernel_centers_candidate_outliers.append(kc)
        kernel_centers_candidate_outliers.sort(key=lambda item: kernel_centers[item].outlier_score_upper_bound, reverse=True)
    

In [145]:
def detect_local_outliers_points():
    outliers = list()

    for kc in kernel_centers_candidate_outliers:
        updated_kc = kc in kernel_centers_required_update
        for data_record in kernel_centers[kc].data:
            if updated_kc or data_record.last_update_window != window_count - 1:
                data_record.previous_local_density = data_record.local_density
                data_record.update_local_density()

            data_record.last_update_window = int(window_count)
            data_record.outlier_score = (data_record.local_density - kernel_centers[kc].local_density_mean) / kernel_centers[kc].local_density_sd
            
            if len(outliers) < expected_outlier_number:
                outliers.append(data_record)
                outliers.sort(key=lambda dr: dr.outlier_score, reverse=True)
           
            elif expected_outlier_number > 0 and data_record.outlier_score <= outliers[0].outlier_score:
                outliers.pop(0)
                outliers.append(data_record)
                outliers.sort(key=lambda dr: dr.outlier_score, reverse=True)

    outliers.sort(key=lambda dr: dr.outlier_score, reverse=False)
    return outliers

In [146]:
# Main workflow

result = list()
oscore = dict()
outliers_dump = dict()
window_dump = dict()

start_time = time.time()
time_deduction = 0

while data_pointer + slide_size <= data.shape[0]:
    slide_start_time = time.time()

    kernel_centers_required_update.clear()

    new_weight = feed_new_slide()

    expired_weight = dict()

    if len(window) < windows_size:
        update_distribution(new_weight, expired_weight)
        continue

    window_count += 1

    if len(window) > windows_size:
        expired_weight = expire_old_slide()

    if fixed_outlier_number:
        expected_outlier_number = fixed_outlier_number

    

    delta_weight_grid = update_distribution(new_weight, expired_weight)

    if window_count == 1 or error_allowance_threshold == 0:
        kernel_centers_required_update.extend(kernel_centers.keys())

    else:
        kernel_centers_required_update.extend(kernel_centers_new)

        for kc in kernel_centers:
            if kernel_centers[kc].removed:
                continue
            if kc in kernel_centers_required_update:
                continue

            for ckc in delta_weight_grid:
                dist = distance_to_kernel_center(ckc, kc)
                if dist <= kernel_centers[kc].nearest_kernel_centers_max_distance:
                    kernel_centers[kc].cumulated_error += abs(delta_weight_grid[ckc] / kernel_centers[kc].nearest_kernel_centers_weight_sum)
                    if kernel_centers[kc].cumulated_error > error_allowance_threshold:
                        kernel_centers[kc].cumulated_error = 0
                        kernel_centers_required_update.append(kc)
                        break
        

    update_changed_kernel_centers()
    detect_local_outliers_cells()
    outliers = detect_local_outliers_points()

    outliers_dump[window_count] = [(o.index, o.array[0], o.array[1], o.outlier_score, o.is_outlier) for o in outliers]

    dump_start_time = time.time()
    window_dump[window_count] = [(r.index, r.array[0], r.array[1], r.outlier_score, r.is_outlier) for r in window]
    
    slide_end_time = time.time()

    time_deduction += slide_end_time - dump_start_time

    log('Slide {0} done, time used: {1:.4f}, active kernels: {2}'.format(window_count, slide_end_time - slide_start_time, len(kernel_centers)))
    sys.stdout.flush()
end_time = time.time()
log('Done, total time used: {0:.4f}'.format(end_time - start_time - time_deduction))


Slide in: 0.0000 to 2.9686
Slide in: 2.9726 to 5.9412
Slide in: 5.9452 to 8.9138
Slide in: 8.9178 to 11.8864
Slide in: 11.8904 to 14.8590
Slide in: 14.8630 to 17.8316
Slide in: 17.8356 to 20.8042
Slide in: 20.8082 to 23.7768
Slide in: 23.7808 to 26.7494
Slide in: 26.7534 to 29.7220
Slide in: 29.7260 to 32.6946
Slide in: 32.6986 to 0.6672
Slide in: 0.6712 to 3.6398
Slide in: 3.6438 to 6.6124
Slide in: 6.6164 to 9.5850
Slide in: 9.5890 to 12.5576
Slide in: 12.5616 to 15.5303
Slide in: 15.5342 to 18.5029
Slide in: 18.5068 to 21.4755
Slide in: 21.4795 to 24.4481
Slide in: 24.4521 to 27.4207
Slide in: 27.4247 to 30.3933
Slide in: 30.3973 to 33.3659
Slide in: 33.3699 to 1.3385
Slide out: 0.0000 to 2.9686
Slide 1 done, time used: 1401.0823, active kernels: 11205
Slide in: 1.3425 to 4.3111
Slide out: 2.9726 to 5.9412
Slide 2 done, time used: 312.9943, active kernels: 11171
Slide in: 4.3151 to 7.2837
Slide out: 5.9452 to 8.9138
Slide 3 done, time used: 305.3842, active kernels: 11190
Slide in: 

In [147]:
fp = open(log_path + '/outliers.log', mode='a')
fp.write(pformat(outliers_dump))
fp.close()

In [148]:
for w in window_dump:
    w_list = list()
    o_list = list()
    for o in outliers_dump[w]:
        o_list.append(o[0])
    for r in window_dump[w]:
        if r[0] in o_list:
            if r[4]:
                typ = 'outlier'
            else:
                typ = 'unlabeled'
        else:
            if r[4]:
                typ = 'undetected'
            else:
                typ = 'normal'
        w_list.append((r[0], r[1], r[2], r[3], typ))
    w_df = pd.DataFrame(w_list, columns=['index', 'time', 'value', 'outlier_score', 'type'])
    fig = px.scatter(w_df, x='time', y='value', hover_data=['outlier_score'], 
                    color='type', render_mode='webgl', opacity=0.5,
                    color_discrete_map={'outlier': 'red', 'normal': 'dodgerblue', 'undetected': 'green', 'unlabeled': 'fuchsia'})
    fig.write_html(log_path + "/window-{0}.html".format(w))
